In [1]:
import pandas as pd
from lifelines import CoxPHFitter
from src.helper_methods import *
from src.pipe_store import *
from src.constants import *
from lifelines import CoxPHFitter

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [2]:
RELEVANT_DESA_BAD = {
    '71TD', '144QL', '70DRA', '80TLR', '70DA', '67F', '76ED', '76ET', '158T', '45EV'
}

# Data Loading

### Total Cohort

In [3]:
path_cohort = '~/UMCUtrecht/KaplanMeier/DESAsurvival.pickle'
path_antibody = '~/UMCUtrecht/20211104_mismatch_ep_db.pickle'
df_abs = pd.read_pickle(path_antibody)
df_cohort = pd.read_pickle(path_cohort)
df = df_abs.merge(df_cohort, on='TransplantID')
cols = [
    'TransplantID', 'DESA_Status', 'EpvsHLA_Donor', '#DESA', 'DESA', 'RecipientAge_NOTR', 
    'DonorAge_NOTR', 'TypeOfDonor_NOTR', 'FailureCode10Y_R', 'GraftSurvival10Y_R', 'IL2rMoAb',
    'CIPHour_DBD', 'CIPHour_DCD' , 'LivingDonorNum', # 'FailureCode10y'
]
df = df[cols]
df.columns

Index(['TransplantID', 'DESA_Status', 'EpvsHLA_Donor', '#DESA', 'DESA',
       'RecipientAge_NOTR', 'DonorAge_NOTR', 'TypeOfDonor_NOTR',
       'FailureCode10Y_R', 'GraftSurvival10Y_R', 'IL2rMoAb', 'CIPHour_DBD',
       'CIPHour_DCD', 'LivingDonorNum'],
      dtype='object')

### All the DESA found

In [23]:
list_set_desa = df[(df.DESA_Status.eq('DESA'))].DESA.values
desas = {ep for desa_set in list_set_desa for ep in desa_set}
print(desas)

{'161D', '9F[A]', '70DR', '97T', '65RNA', '76V', '57V[DQ]', '163LW', '73A', '70QA', '116Y', '74E', '75S', '62GRN', '144K', '11STS', '66IT', '116F', '70DRA', '267QE', '97W', '37YV[DQ]', '55PPD', '116D', '66IS', '56PA', '44RME', '77N[DR]', '143S', '163E', '75IL', '140TV', '67VG', '73AS', '71HS', '71SA', '28H', '31I', '86A', '77T[DQ]', '149AH', '32Y', '167H', '32YN', '166ES', '30H[DR]', '207S', '182N', '98ES', '163LE', '120N', '181T', '130Q', '46VY', '73G', '79GT', '76ANT', '56PD', '62EE', '105S', '138MI', '45GV', '13FE', '16Y', '9T', '71KA', '9F[DQ]', '74EL', '25Q', '55RPD', '96H', '170RH', '98KN', '76VS', '70DA', '77SRN', '17S', '62GE', '40ERV', '76EN', '57V[DR]', '52PQ', '76VDT', '51R', '66KA', '177KT', '71TN', '26F', '66EV', '40E', '253Q', '98Q', '151H', '96HK', '130A', '67I', '55PP', '270C', '149H', '30G[DR]', '90D', '152W', '77R', '193PL', '71E', '71TD', '66NM', '66ER', '4R', '71TTS', '70R', '80I', '71A', '97I', '76ESI', '26L[DR]', '4Q', '181M', '70RT', '189S', '70QK', '248M', '151A

### Total DESA

In [24]:
len(desas)

313

### NUmber of Tx with DESA

In [29]:
len(df[(df.DESA_Status.eq('DESA'))])

439

### NUmber of Tx with clinically relevant DESA

In [31]:
print('Total:', df['DESA'].apply(lambda x: bool(x & RELEVANT_DESA_BAD)).sum())
print('Deceased', df[df.TypeOfDonor_NOTR == 'Deceased']['DESA'].apply(lambda x: bool(x & RELEVANT_DESA_BAD)).sum())
print('Living', df[df.TypeOfDonor_NOTR == 'Living']['DESA'].apply(lambda x: bool(x & RELEVANT_DESA_BAD)).sum())

Total: 76
Deceased 55
Living 21


### Number of DESA per group [total 313]

In [21]:
from collections import defaultdict
from src.constants import Epitope_DB

cls_vs_desa = defaultdict(set)
for ep in desas: 
    for group, epitopes in Epitope_DB.items():
        if ep in epitopes: 
            cls_vs_desa[group].add(ep)
{cls: len(_set) for cls, _set in cls_vs_desa.items()}

{'ABC': 158, 'DR': 89, 'DQ': 57}

### Relevant DESA per group

In [25]:
cls_vs_desa = defaultdict(set)
for ep in RELEVANT_DESA_BAD: 
    for group, epitopes in Epitope_DB.items():
        if ep in epitopes: 
            cls_vs_desa[group].add(ep)

In [28]:
dict(cls_vs_desa)

{'ABC': {'144QL', '158T', '71TD', '76ED', '76ET', '80TLR'},
 'DR': {'67F', '70DA', '70DRA'},
 'DQ': {'45EV'}}

In [ ]:
{
    'ABC': {'144QL', '158T', '71TD', '76ED', '76ET', '80TLR'},
    'DR': {'67F', '70DA', '70DRA'},
    'DQ': {'45EV'}
}

### number of transplants with clinically relevant DESA:

In [36]:
df[df.DESA_Status.eq('DESA')]['DESA'].apply(lambda x: len(x & RELEVANT_DESA_BAD)).value_counts()

0    363
1     53
2      9
3      9
4      4
5      1
Name: DESA, dtype: int64

### number of transplants directed to HLA

In [40]:
get_rel_hla = lambda x: {hla.split('*')[0] for ep, hla in x.items() if ep in RELEVANT_DESA_BAD}
df[(df.DESA_Status == 'DESA')]['EpvsHLA_Donor'].apply(get_rel_hla).value_counts()

{}              363
{B}              36
{DQB1}           28
{DRB1}            6
{DRB5}            3
{DRB1, DRB5}      1
{A}               1
{DQB1, B}         1
Name: EpvsHLA_Donor, dtype: int64

### Tineke Follow-up question: I also need detailed information about all DESAs (loci, number per patient).

In [41]:
get_hla = lambda x: {hla.split('*')[0] for ep, hla in x.items()}
hla_vs_desa = df[df.DESA_Status == 'DESA']['EpvsHLA_Donor'].apply(get_hla).value_counts()
hla_vs_desa

{A}                         83
{B}                         80
{DQB1}                      57
{DRB1}                      36
{A, B}                      20
{C}                         17
{DRB3}                      15
{DRB3, DRB1}                11
{DRB4}                       9
{C, B}                       8
{DQB1, DRB1}                 7
{A, DRB1}                    6
{C, A}                       6
{DRB1, B}                    6
{DRB5}                       5
{DRB4, DRB1}                 5
{DRB1, DRB5}                 5
{DQA1, DQB1}                 5
{A, DQB1}                    4
{A, DRB1, B}                 4
{DRB4, DQB1}                 3
{C, DQB1}                    3
{DQB1, DRB3, DRB1}           2
{DQB1, DRB3}                 2
{DQA1, DQB1, DRB1}           2
{A, C, B}                    2
{A, DQB1, DQA1}              2
{A, DRB3, DRB1}              2
{A, DQB1, DRB1}              2
{DQB1, DRB1, DRB5}           1
{A, DRB4}                    1
{DRB4, DQB1, B}              1
{A, DQB1